In [1]:
import json
import os
import pandas
from scipy.stats import spearmanr

In [2]:
# test score
testgen_df = pandas.read_csv("resources/TestGen_results.csv")
testgen_df['expl_idx'] = testgen_df.run_idx.apply(lambda x: int(x.split('_')[-1]))
high_quality_test_mapping = pandas.DataFrame(testgen_df.groupby(['bug_name', 'expl_idx'])['success'].sum()/10)
testgen_score_df = high_quality_test_mapping.reset_index()
testgen_score_df = testgen_score_df.rename(columns={'success': 'test_score'})
testgen_score_df.head()

,bug_name,expl_idx,test_score
0,Chart_1,1,0.1
1,Chart_1,2,0.5
2,Chart_1,3,0.7
3,Chart_1,4,0.3
4,Chart_1,5,0.1


In [3]:
# apr score
apr_df = pandas.read_csv("resources/APR_results.csv")
apr_df['expl_idx'] = apr_df.patch_idx+1
high_quality_patch_mapping = pandas.DataFrame(apr_df.groupby(['bug_name', 'expl_idx'])['is_positive_patch'].sum()/5)
apr_score_df = high_quality_patch_mapping.reset_index()
apr_score_df = apr_score_df.rename(columns={'is_positive_patch': 'apr_score'})
apr_score_df.head()

,bug_name,expl_idx,apr_score
0,Chart_1,1,0.4
1,Chart_1,2,0.4
2,Chart_1,3,0.4
3,Chart_1,4,0.4
4,Chart_1,5,0.4


In [4]:
# self-rating data
self_rating_df = pandas.read_csv('resources/SelfRating_results.csv')

In [5]:
# AutoFL run data
result_list = []
DATA_DIR = '../results/d4j_autofl_%d/gpt-3.5-turbo-0613/'
for i in range(1, 6):
    for fname in sorted(os.listdir(DATA_DIR % i)):
        if os.path.isdir(os.path.join(DATA_DIR % i, fname)):
            continue
        with open(os.path.join(DATA_DIR % i, fname)) as f:
            run_info = json.load(f)
        bug_name = fname.removeprefix('XFL-').removesuffix('.json')
        grade_results = run_info['buggy_methods']
        if type(grade_results) == str:
            continue
        autofl_success = any(grade_results[gt]['is_found']
                             for gt in grade_results.keys())
        expl_len = len(run_info['messages'][-3]['content'])
        result_list.append({
            'bug_name': bug_name,
            'expl_idx': i,
            'expl_len': expl_len,
            'autofl_success': autofl_success,
        })
result_df = pandas.DataFrame(result_list)

In [6]:
# manual rating data
expl_eval_df = pandas.read_csv('resources/ExplanationQuality.csv')

In [7]:
# combine info
combine_df = pandas.merge(result_df, testgen_score_df, how='outer', on=['bug_name', 'expl_idx'])
combine_df['test_score'] = combine_df.test_score.fillna(0)
combine_df = pandas.merge(combine_df, apr_score_df, how='outer', on=['bug_name', 'expl_idx'])
combine_df['apr_score'] = combine_df.apr_score.fillna(0)
combine_df = pandas.merge(combine_df, self_rating_df, how='outer', on=['bug_name', 'expl_idx'])
expl_char_df = pandas.merge(combine_df, expl_eval_df, how='outer', on=['bug_name', 'expl_idx'])
expl_char_df = expl_char_df[expl_char_df.Exists == True]

In [8]:
# add additional labels
expl_char_df['Wrong'] = (expl_char_df.Accurate == False) & (expl_char_df.Misleading == True)
expl_char_df['Bland'] = (expl_char_df.Accurate == False) & (expl_char_df.Misleading == False)

In [9]:
# variables of interest
table_quality_measures = [
    'Accurate', 'Wrong', 'Useful', 'Bland', 'autofl_success']
table_predict_measures = [
    'test_score', 'apr_score', 'GPT_useful', 'expl_len']

In [10]:
print('Spearman Correlation Table')
table_data = []
for qkey in table_quality_measures:
    qkey_corrs = {'name': qkey}
    for pkey in table_predict_measures:
        corr = spearmanr(
            expl_char_df[qkey],
            expl_char_df[pkey],
        )
        qkey_corrs[pkey] = corr[0]
    table_data.append(qkey_corrs)
table_df = pandas.DataFrame(table_data)
table_df.head()

Spearman Correlation Table


,name,test_score,apr_score,GPT_useful,expl_len
0,Accurate,0.235818,0.194671,0.375900,0.300970
1,Wrong,0.040855,-0.064387,0.326664,0.327153
2,Useful,0.263538,0.194270,0.237105,0.158562
3,Bland,-0.236484,-0.110560,-0.602640,-0.539122
4,autofl_success,0.273725,0.492391,0.143723,0.152852


In [11]:
print('Spearman Correlation Table (Bug-Aggregated)')
table_data = []
for qkey in table_quality_measures:
    qkey_corrs = {'name': qkey}
    for pkey in table_predict_measures:
        bug_average_pkey = expl_char_df.groupby('bug_name')[pkey].mean()
        bug_any_qkey = expl_char_df.groupby('bug_name')[qkey].any()
        corr = spearmanr(
            bug_any_qkey,
            bug_average_pkey,
        )
        qkey_corrs[pkey] = corr[0]
    table_data.append(qkey_corrs)
table_df = pandas.DataFrame(table_data)
table_df.head()

Spearman Correlation Table (Bug-Aggregated)


,name,test_score,apr_score,GPT_useful,expl_len
0,Accurate,0.263591,0.378972,0.449137,0.211693
1,Wrong,-0.032382,-0.150237,0.154724,0.057587
2,Useful,0.244773,0.335074,0.229434,-0.040958
3,Bland,-0.238547,-0.130931,-0.172254,-0.123462
4,autofl_success,0.279275,0.563470,0.307735,0.098383


In [12]:
print('Spearman Correlation Table (Bug-Controlled)')
table_data = []
for qkey in table_quality_measures:
    qkey_corrs = {'name': qkey}
    for pkey in table_predict_measures:
        bug_average_pkey = expl_char_df.groupby('bug_name')[pkey].mean()
        bug_any_qkey = expl_char_df.groupby('bug_name')[qkey].any()
        adjusted_pkey = []
        actually_qkey = []
        for _, row in expl_char_df.iterrows():
            if not row.Exists:
                continue
            average_pkey_for_bug = bug_average_pkey[row.bug_name]
            adjusted_pkey.append(row[pkey]-average_pkey_for_bug)
            actually_qkey.append(row[qkey])
        corr = spearmanr(
            actually_qkey,
            adjusted_pkey,
        )
        qkey_corrs[pkey] = corr[0]
    table_data.append(qkey_corrs)
table_df = pandas.DataFrame(table_data)
table_df.head()

Spearman Correlation Table (Bug-Controlled)


,name,test_score,apr_score,GPT_useful,expl_len
0,Accurate,0.071364,0.003158,0.292991,0.230642
1,Wrong,0.082870,-0.001171,0.330693,0.301972
2,Useful,0.095520,-0.108542,0.202789,0.147343
3,Bland,-0.132405,-0.001684,-0.535368,-0.457381
4,autofl_success,-0.046269,0.046109,0.040097,0.147168
